# Edge detection

There is a lot of methods to detect the edges of an image. Each have its own advantages and drawbacks. We tested several of them and will use the one that seems the most apporpiate to us for each category of image.

In [1]:
import cv2
import numpy as np
import tools
import matplotlib.cm as cm   
from ipywidgets import interact, fixed

from image import *
from edge import *
from line import *

from notebook_utils import *

SyntaxError: invalid syntax (notebook_utils.py, line 44)

In [ ]:
img = load_gray_img("img/boat.png")

grad_sobel = sobel_edge(img)
grad_naive = naive_gradient(img)
grad_scharr = scharr_edge(img)
grad_beucher = beucher_edge(img)
grad_canny = canny_edge(img)

tools.multiPlot(2, 3, 
        (img, grad_naive, grad_sobel, grad_scharr, grad_beucher, grad_canny),
        ('Original Image', 'Naive Gradient', 'Sobel', 'Scharr', 'Beucher Gradient', 'Canny'),
        cmap_tuple=(cm.gray, cm.gray, cm.gray, cm.gray, cm.gray, cm.gray))

## Sobel Edge

In [ ]:
interact(update_edges, method = "sobel",
             image = ["building", "sudoku", "soccer", "road", "pcb"],
             low_filter_type = ["uniform", "median", "gaussian"],
             low_filtering_kernel_size = (1, 9, 2) ,
             high_filter_type = ["uniform", "median", "gaussian"],
             high_filtering_kernel_size = (1, 9, 2),
             strength = (1.5, 5.0, 0.5),
             block_size = (3, 27, 4),
             constant = (1, 11, 2),
             edge_threshold = (0, 100, 10))

## Scharr Edges

In [ ]:
interact(update_edges, method = "scharr",
             image = ["building", "sudoku", "soccer", "road", "pcb"],
             low_filter_type = ["uniform", "median", "gaussian"],
             low_filtering_kernel_size = (1, 9, 2) ,
             high_filter_type = ["uniform", "median", "gaussian"],
             high_filtering_kernel_size = (1, 9, 2),
             strength = (1.5, 5.0, 0.5),
             block_size = (3, 27, 4),
             constant = (1, 11, 2),
             edge_threshold = (0, 100, 10))

## Naive Gradient

In [ ]:
interact(update_edges, method = "naive_gradient",
             image = ["building", "sudoku", "soccer", "road", "pcb"],
             low_filter_type = ["uniform", "median", "gaussian"],
             low_filtering_kernel_size = (1, 9, 2) ,
             high_filter_type = ["uniform", "median", "gaussian"],
             high_filtering_kernel_size = (1, 9, 2),
             strength = (1.5, 5.0, 0.5),
             block_size = (3, 27, 4),
             constant = (1, 11, 2),
             edge_threshold = (0, 100, 10))

## Beucher Gradient

In [ ]:
interact(update_edges, method = "beucher",
             image = ["building", "sudoku", "soccer", "road", "pcb"],
             low_filter_type = ["uniform", "median", "gaussian"],
             low_filtering_kernel_size = (1, 9, 2) ,
             high_filter_type = ["uniform", "median", "gaussian"],
             high_filtering_kernel_size = (1, 9, 2),
             strength = (1.5, 5.0, 0.5),
             block_size = (3, 27, 4),
             constant = (1, 11, 2),
             edge_threshold = (0, 100, 10))

## Canny Edges 

In [ ]:
interact(update_edges, method = "canny",
             image = ["building", "sudoku", "soccer", "road", "pcb"],
             low_filter_type = ["uniform", "median", "gaussian"],
             low_filtering_kernel_size = (1, 9, 2) ,
             high_filter_type = ["uniform", "median", "gaussian"],
             high_filtering_kernel_size = (1, 9, 2),
             strength = (1.5, 5.0, 0.5),
             block_size = (3, 27, 4),
             constant = (1, 11, 2),
             edge_threshold = (0, 100, 10),
             low_threshold= (0, 100, 10), high_threshold= (200, 250, 10), 
             aperture_size= (1, 11, 2))

## Stacked

In [ ]:
interact(update_edges, method = "stacked",
             image = ["building", "sudoku", "soccer", "road", "pcb"],
             low_filter_type = ["uniform", "median", "gaussian"],
             low_filtering_kernel_size = (1, 9, 2) ,
             high_filter_type = ["uniform", "median", "gaussian"],
             high_filtering_kernel_size = (1, 9, 2),
             strength = (1.5, 5.0, 0.5),
             block_size = (3, 27, 4),
             constant = (1, 11, 2),
             edge_threshold = (0, 100, 10))